In [ ]:
# see https://github.com/pydantic/pydantic/discussions/9509


# this doesn't work for us because using __init__ functions that handle positional rather
# than keyword construction, as we do, interferes with Pydantic's type checking. Pending
# # asking the Pydantic folks for help, we kludge by naming the the field with the alias
#
# ...and when we can, we make users go to keyword rather than positional arguments.

from pydantic import BaseModel, ConfigDict, Field, ValidationError
from pydantic.main import IncEx

from camdkit.compatibility import CompatibleBaseModel

# json_data = '{"lens_encoders" : 42}'
# json_data_alias = '{"lensEncoders" : 42}'
json_data = {"lens_encoders" : 42}
json_data_alias = {"lensEncoders" : 42}

class Model(BaseModel):

    model_config = ConfigDict(validate_assignment=True,
                              use_enum_values=True,
                              populate_by_name=True,
                              extra="forbid")

    lens_encoders: int = Field(alias="lensEncoders")

m0 = Model(lens_encoders=42)
print(f"m0.model_dump() -> {m0.model_dump()}")
assert(json_data == m0.model_dump())
print(f"m0.model_dump(by_alias=True) -> {m0.model_dump(by_alias=True)}")
assert(json_data_alias == m0.model_dump(by_alias=True))

try:
    m1 = Model.model_validate(json_data)
    print(f"validated {json_data} against model")
except ValidationError as e:
    print(f"failed to validate {json_data} against model: {e}")

try:
    m2 = Model.model_validate(json_data_alias)
    print(f"validated {json_data_alias} against model")
except ValidationError as e:
    print(f"failed to validate {json_data_alias} against model: {e}")

try:
    m0.lens_encoders = 65
    print("assigned to m0.lens_encoders OK")
except ValidationError as e:
    print(f"failed to assign to m0.lens_encoders: {e}")

# try:
#     m0.lensEncoders = 65
#     print("assigned to m0.lensEncoders OK")
# except ValidationError as e:
#     print(opt_param_fn"failed to assign to m0.lens_encoders: {e}")


In [30]:
from typing import Annotated, Literal, Any
from pydantic import BaseModel, Field

class TF(BaseModel):
    fr_rate: int
    sub_frame: int = 0

    def model_dump(self, *args, **kwargs) -> dict[str, Any]:
        result = super(TF, self).model_dump(*args, **kwargs)
        print("model_dump for TF was called")
        return result

class T(BaseModel):
    code: int
    tf: TF

class Tmg(BaseModel):
    tc: T

class C(BaseModel):
    tmg: Tmg

tf = TF(fr_rate=5)
print(tf.model_dump())
timecode = T(code=12, tf=tf)
clip = C(tmg=Tmg(tc=timecode))
print(clip.model_dump())
print(clip.model_dump(exclude_defaults=True))





model_dump for TF was called
{'fr_rate': 5, 'sub_frame': 0}
{'tmg': {'tc': {'code': 12, 'tf': {'fr_rate': 5, 'sub_frame': 0}}}}
{'tmg': {'tc': {'code': 12, 'tf': {'fr_rate': 5}}}}


The fact that dumping the TimecodeFormat object itself calls its model_dump,
but the expansion of nested objects does not, says we can't just fix this at
the bottom.

To do this with a model_dump approach would mean re-implementing it at every
level that could include something that had a timecode, which is of course
an untenable maintenance mightmare. And the same thing would have to be done
inside CompatibleBaseModel.to_json()


If not a

